In [1]:
#load the dateset
import pandas as pd

#load the dataset, no header
df = pd.read_csv('lercio_headlines.csv', header=None)
df.head()

# create a txt file with the headlines
with open('lercio_headlines.txt', 'w') as f:
    for line in df[0]:
        f.write(line.lower() + '.\n')

The lercio dataset is more or less 3 times smaller than the shakespeare dataset that is the baseline for the tests

In [4]:
#find the mean length of the headlines
df[0].str.len().mean()

70.49661225746843

In [5]:
import os

path = "./lercio_headlines_base.txt"

options =\
"--model            gru   " +\
"--n_epochs         700    " +\
"--print_every      50   " +\
"--hidden_size      32    " +\
"--n_layers         4     " +\
"--learning_rate    0.01  " +\
"--chunk_len        100   " +\
"--batch_size       1000   " +\
"--cuda                   " 

command = f"python ./char-rnn-pytorch/train.py {path} {options}"


os.system(command)

usage: train.py [-h] [--filename FILENAME] [--model MODEL]
                [--n_epochs N_EPOCHS] [--print_every PRINT_EVERY]
                [--hidden_size HIDDEN_SIZE] [--n_layers N_LAYERS]
                [--learning_rate LEARNING_RATE] [--chunk_len CHUNK_LEN]
                [--batch_size BATCH_SIZE] [--shuffle] [--cuda]
train.py: error: unrecognized arguments: ./lercio_headlines_base.txt


512

In [6]:
import os
path = "../lercio_headlines_base.pt"

options =\
'--prime_str   " "    ' +\
"--predict_len    100    " +\
"--cuda               " 
#"--temperature    100     " +


command = f"python ./char-rnn-pytorch/generate.py {path} {options}"

os.system(command)

/home/matteo/anaconda3/envs/ispr/lib/python3.10/site-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/matteo/anaconda3/envs/ispr/lib/python3.10/site-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
Traceback (most recent call last):
  File "/home/matteo/ispr/midterm_3/./char-rnn-pytorch/generate.py", line 54, in <module>
    decoder = torch.load(args.filename)
  File "/home/matteo/anaconda3/envs/ispr/lib/p

256